# Recommendations and Matching Basics using Langchain and Vertex GenAI 

## Summary

This notebook Illustrates how to create basic recommendations and generate matching using langchain and Vertex GenAI LLM models

#### Overview


#TODO: Brief description here

#### Setup 

In [ ]:
!pip install -U google-cloud-aiplatform langchain --user

In [ ]:
import json 

import vertexai
from vertexai.preview.language_models import TextGenerationModel

import langchain
from pydantic import BaseModel
from langchain.llms.base import LLM
from langchain import PromptTemplate, LLMChain
from langchain.llms import VertexAI

Create custom vertex wrapper class to call LLM

In [ ]:
# LLM custom wrapper

class VertexLLMTextExractor(LLM):
    model: TextGenerationModel
    predict_kwargs: dict

    def __init__(self, model, **predict_kwargs):
        super().__init__(model=model, predict_kwargs=predict_kwargs)

    @property
    def _llm_type(self):
        return 'VertexLLM'

    def _call(self, prompt, stop=None):
        result = self.model.predict(prompt, **self.predict_kwargs)
        return str(result)

    @property
    def _identifying_params(self):
        return {}

# Call llm model

model = TextGenerationModel.from_pretrained("text-bison@001")
parameters = {
    "max_output_tokens": 1024,
    "temperature": 0.2,
    "top_k": 40,
    "top_p": 0.8,
}

llm = VertexLLMTextExractor(
  model,
  **parameters
)

In [ ]:
# Set of helper functions

# Function to get schema
def get_response_schema(chain: str):
    
    from langchain.output_parsers import StructuredOutputParser, ResponseSchema
    from langchain.prompts import HumanMessagePromptTemplate
    
    # Define recommended color & brand schema
    recommendation_response_schema = [
        ResponseSchema(name="recommended_brand_name", description="recommended brand name from llm output"),
        ResponseSchema(name="recommended_color_name", description="recommended color name from llm output")
    ]
    
    # Format response intructions
    response_schema_output_parser = StructuredOutputParser.from_response_schemas(recommendation_response_schema)
    recommendation_response_format_instructions = response_schema_output_parser.get_format_instructions()
    
    # Define matched color schema
    matches_response_schema = [
        ResponseSchema(name="recommended_brand_name", description="given recommended brand name"),
        ResponseSchema(name="recommended_color_name", description="given recommended color name"),
        ResponseSchema(name="matched_brand_name", description="matched brand name for given recommended_color_name and recommended_brand_name combination"),
        ResponseSchema(name="matched_color_name", description="matched color name for given recommended_color_name and recommended_brand_name combination"),
        ResponseSchema(name="matched_uri", description="color uri of matched color name for given recommended_color_name and recommended_brand_name combination")
    ]
    
    # Format response intructions
    matches_response_schema_output_parser = StructuredOutputParser.from_response_schemas(matches_response_schema)
    matches_response_format_instructions = matches_response_schema_output_parser.get_format_instructions()
    
    if chain == 'recommend':
        return recommendation_response_format_instructions
    elif chain == 'match':
        return matches_response_format_instructions
    else:
        pass
    
        
# Function to generate prompt template
def generate_prompt(chain: str, input_prompt_text: str):
    
    format_intruction = get_response_schema(chain)
    
    if chain == 'recommend':
        
        # Create prompt template
        prompt = PromptTemplate(
            input_variables=["user_input"],
            partial_variables={"format_instructions": format_intruction},
            template=recommend_template
        )

        color_recommendations_promptValue = prompt.format(user_input=input_prompt_text)        
        return prompt
        
    elif chain == 'match':
        matches_prompt = PromptTemplate(
            input_variables=["recommended_brand_name, recommended_color_name"],
            partial_variables={"format_instructions": format_intruction},
            template=match_template
        )
        return matches_prompt

    else:
        return None
        
    

In [ ]:
# Function to recommend and match colors

def recommend_and_matches(input_prompt_text: str):
    
    import json
    from langchain.chains import LLMChain, SimpleSequentialChain
    
    # Simple sequential chain
    # Holds recommended colors from user input response
    recommended_color_chain = LLMChain(llm=llm, prompt=generate_prompt('recommend', input_prompt_text))

    # Holds matchee colors from recommended colors
    matched_color_chain = LLMChain(llm=llm,prompt=generate_prompt('match', input_prompt_text))
    
    # Build final chain
    overall_chain = SimpleSequentialChain(chains=[recommended_color_chain, matched_color_chain], verbose=False)
    colors = overall_chain.run(user_input)
    
    json_colors = json.loads(colors.strip('```json```'))
    
    return json_colors #(json.dumps(json_colors, indent = 4)) 
    

In [ ]:
recommend_and_matches(user_input)

In [ ]:
print(get_response_schema('recommend'))
print(get_response_schema('match'))      

In [ ]:
print(generate_prompt('recommend',user_input))
print('**************************************')
print(generate_prompt('match',user_input))